In [28]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
from tensorflow.keras.datasets import boston_housing
Train, test = boston_housing.load_data(seed= 111)
train_data, train_labels = Train[0], Train[1]
test_data, test_labels = test[0], test[1]

In [30]:
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

(404, 13)
(404,)
(102, 13)
(102,)


## Adding dummy feature

In [31]:
train_dummy = np.ones(len(train_labels))
test_dummy = np.ones(len(test_labels))
train_data = np.row_stack((train_dummy, train_data.T))
test_data = np.row_stack((test_dummy, test_data.T))
print(train_data.shape)
print(test_data.shape)

(14, 404)
(14, 102)


In [43]:
features = train_data.shape[0]
features

14

# Normal Equation LR

## Computing Weights
Formula to calculate weights is:
$$
w_{lr} = (XX^T)^{-1}XY
$$

In [32]:
def weights_lr(X,y):
    return np.linalg.pinv(X @ X.T) @ X @ y

## Computing Predictions
$$
y^{'}=w^TX
$$

In [33]:
def predict(X,w):
    return w.T @ X

## Computing Loss (RMSE)
$$
loss = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y^{'}_{i}-y_{i})^2}
$$

In [34]:
def RMSE(y, y_pred):
    return np.sqrt(np.mean((y-y_pred)**2))

In [35]:
# Weights for normal equation LR
w_lr = weights_lr(train_data, train_labels)

In [36]:
np.sum(w_lr)

20.524582465710353

In [37]:
# Predictions for normal equation LR
pred_lr_train = predict(train_data, w_lr)
np.mean(pred_lr_train)

22.30915841584466

In [38]:
# training loss for normal equation LR
RMSE(train_labels, pred_lr_train)

4.552387969840813

In [39]:
# test loss for normal equation LR
pred_lr_test = predict(test_data, w_lr)
RMSE(test_labels, pred_lr_test)

5.327662216175009

# Gradient Descent

## Computing Weights
$$
w^{t+1}=w^{t}-\eta^{t}(2XX^{t}w^{t}-2XY)
$$

In [47]:
def weights_gd(X, y, w, eta, iters):
    for i in range(iters):
        w = w - eta*(2*X @ X.T @ w - 2*X @ y)
    return w

In [49]:
w_gd = weights_gd(train_data, train_labels, np.zeros(features), 1e-10, 100)
np.sum(w_gd)

0.058959061195902614

In [51]:
# training error for GD
pred_gd_train = predict(train_data, w_gd)
RMSE(train_labels, pred_gd_train)

11.13727323702196

In [52]:
pred_gd_test = predict(test_data, w_gd)
RMSE(test_labels, pred_gd_test)

10.964491250062146

# Stochastic Gradient Descent
Run Gradient Descent Algorithm for fixed number of iterations on random sample of data points.

In [55]:
def weights_sgd(X, y, eta, iters):
    w = np.zeros(X.shape[0])
    for i in range(iters):
        rng = np.random.default_rng(seed = i)
        sample_indices = rng.integers(0, X.shape[1], X.shape[1]//5)
        X_sample = X[:,sample_indices]
        y_sample = y[sample_indices]
        w = w - eta*(2*X_sample @ X_sample.T @ w - 2*X_sample @ y_sample)
    return w

In [57]:
w_sgd = weights_sgd(train_data, train_labels, 1e-8, 1000)
np.sum(w_sgd)

0.10124206187312637

In [58]:
pred_sgd_train = predict(train_data, w_sgd)
RMSE(train_labels, pred_sgd_train)

8.6246359251934

In [59]:
pred_sgd_test = predict(test_data, w_sgd)
RMSE(test_labels, pred_sgd_test)

8.352305624451189

# Kernel Regression

In [86]:
# Training Data Setup
rng = np.random.default_rng(seed = 101)
X = np.arange(-2, 2, 0.01).reshape(-1, 1)
y = X**3 + rng.normal(0, 1, X.shape[0]).reshape(-1, 1)

In [87]:
y = y.reshape(-1,1)
y.shape

(400, 1)

In [88]:
X = np.row_stack((np.ones(X.shape[0]), X.T))
X.shape

(2, 400)

In [89]:
def predict_kernel(X,y, deg):
    K = (X.T @ X + 1)**deg # polynomial kernel
    alpha = np.linalg.pinv(K) @ y # coefficient
    return K.T @ alpha

In [90]:
np.sum(predict_kernel(X,y,3))

-24.089592244996595

In [84]:
RMSE(y, predict_kernel(X,y,3))

1.0061539382919291

In [91]:
# Test Data
rng = np.random.default_rng(seed = 102)
Xnew = np.arange(-2, 2, 0.03)
ynew = Xnew**3 + rng.normal(0, 1.5, Xnew.shape[0])
X_test = np.column_stack((np.ones(Xnew.shape[0]), Xnew.reshape(-1, 1))).T
y_test = ynew.reshape(-1, 1)

In [92]:
RMSE(y_test, predict_kernel(X_test, y_test, 3))

1.5480083925033956